**As a newbie to data world, I thought to port one of the solution provided for this problem in Python. Original solution was in R by Philip. It was a great learning experience and at the same time give me a comparison of which is working well to process and explore data
R seems to be winner as I am yet to find the solution to visualize treemap in python. It seems there is no package available.  Learned a lot on how to use python syntax. Thankful to the community for posting such cool**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
import matplotlib.pyplot as plt
# Any results you write to the current directory are saved as output.

In [2]:
#Let's explore datasets
#aisles.csv
aisles = pd.read_csv('../input/aisles.csv')
#departments.csv
departments = pd.read_csv('../input/departments.csv')
#order_products__prior.csv
order_products_prior = pd.read_csv("../input/order_products__prior.csv")
#order_products__train.csv
order_products_train = pd.read_csv("../input/order_products__train.csv")
#orders.csv
orders = pd.read_csv("../input/orders.csv")
#products.csv
products = pd.read_csv('../input/products.csv')
#sample_submission.csv
sample_submission = pd.read_csv('../input/sample_submission.csv')

In [3]:
aisles.head()

In [4]:
departments.head()

In [5]:
order_products_prior.head()

In [6]:
order_products_train.head()

In [7]:
orders = orders.fillna(-1)
orders.head()

## Day of Week ##
The Histogram below suggests that most shopping happens on Day 0 and Day 1.

In [8]:

#unique DOW values
orders.order_dow.unique()
#what is the frequency of the orders according to days
n, bins, patches = plt.hist(orders.order_dow, 13, facecolor="red", alpha=.75, align='mid')
plt.xlabel("Day of Week")
plt.ylabel("Orders Count")
plt.title("When do people buy?")
plt.show()


## Hour of The Day ##
**What time most people buy?**
The results below shows the customers are buying between 8am to 6pm

In [9]:
orders.order_hour_of_day.unique()
n, bins, patches = plt.hist(orders.order_hour_of_day, 47, facecolor="red", alpha=.75, align='mid')
plt.xlabel("Hour of Day")
plt.ylabel("Orders Count")
plt.title("When do people buy in a Day?")
plt.show()

## How Many Items People Buy per order##

In [10]:
order_products_cart_prior = order_products_prior.groupby('order_id')
n_items = order_products_cart_prior['add_to_cart_order'].max()
n, bins, patches = plt.hist(n_items, 100, facecolor="red", alpha=.75, align='mid')
plt.show()

In [11]:
order_products_cart_prior.head()

In [12]:
order_products_cart_prior = order_products_train.groupby('order_id')
n_items_train = order_products_cart_prior['add_to_cart_order'].max()
n, bins, patches = plt.hist(n_items_train, 100, facecolor="red", alpha=.75, align='mid')
plt.show()

## What are the best seller products ##
Banana is the clear winner

In [13]:
products.head()

In [14]:
sample_submission.head()

In [15]:
tmp = order_products_prior.groupby('product_id')['add_to_cart_order'].count().to_frame()
tmp.head()

In [16]:
merged = pd.merge(products, pd.DataFrame(tmp, columns=['add_to_cart_order']), 
                  left_on='product_id', right_index=True)
merged.head()

In [17]:
merged.sort_values('add_to_cart_order', ascending=False)

## How often do people order the same items again? ##

In [18]:
reordered_products_train = pd.DataFrame({'count' : order_products_train.groupby( ['reordered'] ).size()}).reset_index()
newcol = reordered_products_train['count']/reordered_products_train['count'].sum()
reordered_products_train = reordered_products_train.assign(proportion = newcol )
reordered_products_train
#tmp_reorder = order_products_train.groupby('reordered')['order_id'].count().to_frame()
#tmp_reorder.reset_index()

##Most Often Reordered Product

In [19]:
most_reordered_product = pd.DataFrame({'product_id':order_products_train['product_id'],
                                     'proportion':order_products_train.groupby('product_id')['reordered'].mean(),
                                      'n': order_products_train.groupby('product_id')['reordered'].count()})
merged_reordered = pd.merge(products, pd.DataFrame(most_reordered_product, columns=['n', 'product_id', 'proportion']), 
                  left_on='product_id', right_index=True).reset_index()
merged_reordered.head()
merged_reordered = merged_reordered[merged_reordered['n'] > 40.0]
merged_reordered.sort_values('proportion', ascending=False).head()

## Which Item do People put in the Cart first (Train data) ##

In [20]:
product_by_id_order = pd.DataFrame({'count':order_products_train.groupby(['product_id', 'add_to_cart_order']).size()})
new_col = product_by_id_order['count']/product_by_id_order['count'].sum()
product_by_id_order = product_by_id_order.assign(pct=new_col)
product_by_id_order = product_by_id_order.reset_index()
product_by_id_order.head()

In [21]:
merged_product_reordered = pd.merge(products, product_by_id_order,
                                   on='product_id', how="left")
merged_product_reordered.reset_index()
merged_product_reordered.head()
merged_product_reordered = merged_product_reordered[merged_product_reordered['add_to_cart_order'] == 1]
merged_product_reordered = merged_product_reordered[merged_product_reordered['count'] > 10]
merged_product_reordered.sort_values('pct', ascending=False, axis=0)

## Association between time of last order and probability of reorder ##

In [22]:
order_time_join = pd.merge(order_products_train, orders, on="order_id")
order_time_join.head()

In [23]:
order_time_groupby_reorder = pd.DataFrame({'mean_reorder': order_time_join.groupby(['days_since_prior_order'])['reordered'].mean()})
order_time_groupby_reorder = order_time_groupby_reorder.reset_index().head()
plt.ylabel('Mean Reorder')
plt.xlabel('Days Since Prior Order')
plt.bar(order_time_groupby_reorder['days_since_prior_order'], 
        order_time_groupby_reorder['mean_reorder'], color='red')
plt.show()

## Association between number of orders and probability of reordering ##

In [24]:
product_by_reordered = pd.DataFrame({'mean_reorder': order_products_train.groupby(['product_id'])['reordered'].mean(),
                                    'count': order_products_train.groupby(['product_id'])['order_id'].count()})
product_by_reordered = product_by_reordered.reset_index()

In [25]:
plt.xlim([0,2000])
plt.yticks([0,0.5,1.0,1.5])
plt.plot(product_by_reordered['count'],product_by_reordered['mean_reorder'] , 'ro')
plt.show()


##Visualizing the Product Portfolio (treemap not on python ?) ##

##Exploring Customer Habits##
Let's look at the customers who purchase the same products again all the time.

In [26]:
order_groupby_orderid = pd.DataFrame({'order_id': order_products_prior.order_id,
                                      'mean_reorder': order_products_prior.groupby(['order_id'])['reordered'].mean(),
                                     'product_id':order_products_prior.product_id
                                     })
order_groupby_orderid.dtypes

In [27]:
order_groupby_orderid.head()

In [28]:
#order_groupby_orderid = order_groupby_orderid.dropna()
order_groupby_orderid.count()

In [29]:
# Let's Join the table to add user_id info 
merged_df = pd.merge(order_groupby_orderid, pd.DataFrame(orders, columns=['order_number', 'user_id']), 
                  left_on='product_id', right_index=True)

In [30]:
merged_df.reset_index()
merged_df.head()

In [36]:
merged_df = merged_df.dropna()
merged_df = merged_df[merged_df['mean_reorder']==1]
merged_df = merged_df[merged_df['order_number']>2]

In [37]:
merged_df.head()

In [47]:
user_id_with_meanreorder = merged_df.groupby(['user_id']).count()

In [ ]:
#user_id_with_meanreorder.head(20)
user_id_with_meanreorder.sort_values(by='mean_reorder', ascending=False)

In [52]:
newcol = user_id_with_meanreorder['mean_reorder']/user_id_with_meanreorder['order_number']
user_id_with_meanreorder = user_id_with_meanreorder.assign(percent_equal = newcol)

In [ ]:
user_id_with_meanreorder = user_id_with_meanreorder.reset_index()

In [ ]:
user_id_with_meanreorder.head()

In [ ]:
user_id_with_meanreorder.sort_values(by='mean_reorder', ascending=False)

## to be continued...

some bug in calculation as result don't match original solution